# Chapter 6 - Example: Ocean Data 
###  Ocean area with temperature above a given threshold

In this chapter we exemplify the use of Sea Surface Temperature (SST) data in the cloud. 

This example analyze a time series from an area of the ocean or a point. If an area, averages SST values. Then it analyze the time series to assess when SST is above a given threshold. This could be used to study marine heatwaves, or use a threshold relevant for a marine species of interest.

In [ ]:
# Import libraries
import warnings  # this library helps to make your code execution less messy

warnings.simplefilter("ignore")  # filter some warning messages
import dask
import fsspec  # these libraries help reading cloud data
import hvplot.pandas  # this library helps to make interactive plots
import hvplot.xarray
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import s3fs
import xarray as xr
from dask.distributed import Client, performance_report, progress

xr.set_options(keep_attrs=True)

In [ ]:
# Input parameters
# Select either a range of lat/lon or a point. If a point, set both entries to the same value
latr = [
    19,
    20,
]  # make sure lat1 > lat2 since no test is done below to simplify the code
lonr = [-158, -157]  # lon1 > lon2, range -180:180. resolution daily 1km!
# time range. data range available: 2002-06-01 to 2020-01-20. [start with a short period]
dater = ["2012-01-01", "2019-12-31"]  # dates on the format 'YYYY-MM-DD' as string

***
## We are going to use the Multi-Scale Ultra High Resolution (MUR) Sea Surface Temperature (SST) data set
### Stored the Amazon (AWS) Cloud. For more info and links to the data detail and examples, see: https://registry.opendata.aws/mur/

This dataset is stored in 'zarr' format, which is an optimized format for the large datasets and the cloud. It is not stored as one 'image' at a time or a gigantic netcdf file, but in 'chunks', so it is perfect for extracting time series.

First, we open the dataset and explore it, but we are not downloading it yet.

In [ ]:
# first determine the file name using:
# the s3 bucket [mur-sst], and the region [us-west-2], and the folder if applicable [zarr-v1]
file_location = "https://mur-sst.s3.us-west-2.amazonaws.com/zarr-v1"

# open a zarr file using xarray, similar to open_dataset but only reads the metadata
ds_sst = xr.open_zarr(file_location, consolidated=True)

# look at the datarray structure, description and attributes
ds_sst
# click on the page icon on the far right of each varraible/coordinate for information

## Now that we know what the file contains, we select our data (region and time), operate on it if needed (if a region, average), and download only that data 
It takes a while, given the high resolution of the data. So, be patient.... and if you're only testing, might want to choose a small region and a short time period first. 

In [ ]:
sst = (
    ds_sst["analysed_sst"]
    .sel(
        time=slice(dater[0], dater[1]),
        lat=slice(latr[0], latr[1]),
        lon=slice(lonr[0], lonr[1]),
    )
    .mean(dim={"lat", "lon"}, skipna=True)
)  # .load() # skip 'not a number' values and keep attributes
sst

In [ ]:
# decide if a point or a region was given.
if (latr[0] == latr[1]) | (lonr[0] == lonr[1]):  # If we give it only one point
    sst = (
        ds_sst["analysed_sst"]
        .sel(time=slice(dater[0], dater[1]), lat=latr[0], lon=lonr[0])
        .load()
    )
else:  # if we give it an area, it extract the area and average SST over the area and returns a time series of SST
    sst = (
        ds_sst["analysed_sst"]
        .sel(
            time=slice(dater[0], dater[1]),
            lat=slice(latr[0], latr[1]),
            lon=slice(lonr[0], lonr[1]),
        )
        .mean(dim={"lat", "lon"}, skipna=True)
        .load()
    )  # skip 'not a number' values

sst = sst - 273.15  # transform Kelving to degrees Celsius
sst.attrs["units"] = "celsius"  # don't forget to reset attributes to new units

sst.to_netcdf(
    "sst_example.nc"
)  # we are saving the data. if we need to come back to analyze the same data, we do not have to acquire it again from the cloud.
sst  # take a peak

***
### *Execute the next cell only if your reading the data from a file - either no access to cloud, or not want to keep reading from it. Skip otherwise. (No problem if you executed it by mistake).*

In [ ]:
sst = xr.open_dataset("./sst_example.nc")  # read a netcdf
sst.close()
sst = sst.analysed_sst  # sst has the same info downloaded above

***
## Let's plot the data using two different libraries.
#### - `matplotlib` that we already learn. It makes static and very nice figures that can be customized.
#### - `hovplot` is a more interactive library for web display. it provides you with the data details when you hover your cursor over the figure. Very nice for inspecting the data.

In [ ]:
# matplotlib method#
print("matplotlib")
sst.plot()  # this is all you need

# all the stuff here to make it look nice. try commenting them out
plt.xlabel("Year")
plt.title("Location: " + str(latr) + "$^\circ$N, " + str(lonr) + "$^\circ$W")
plt.grid(True, alpha=0.3)
plt.show()

# hovplot method #
print("hovplot")
df = pd.DataFrame(data=sst.data, index=sst.time.data, columns=["SST (C)"])
df.index.name = "Date"
df.hvplot(grid=True)

***
## Now, let's analyze our data.
#### First, the a basic climatology and anomalies, and plotting using `hovplot`.

In [ ]:
# Calculate the climatology
sst_climatology = sst.groupby("time.dayofyear").mean("time", skipna=False)  # Group by day, all years. skipna ignore missing values (NaN=Not a Number)
sst_climstd = sst.groupby("time.dayofyear").std("time", skipna=False)  # Calculate standard deviation.

# Creates a pandas dataframe (a table-like structure) to plot easily using hvplot.
df = pd.DataFrame(    data=sst_climatology.data, index=sst_climatology.dayofyear.data, columns=["SST (C)"])
df["+Std"] = (    sst_climstd.data + sst_climatology.data)  # add standard deviation time series +/-
df["-Std"] = -sst_climstd.data + sst_climatology.data
df.index.name = "Day of Year"
df.hvplot(    color=["k", "grey", "grey"], grid=True, title="SST Climatology")  # plot the climatology (black, and the standard deviation in grey)

In [ ]:
# Calculate the anomalies
sst_anomaly = sst.groupby("time.dayofyear") - sst_climatology
sst_anomaly_monthly = sst_anomaly.resample(time="1MS", loffset="15D").mean(    skipna=False)  # calculate monthly anomalies/smoothing

# Make a pandas dataframe for easy plotting with hvplot
df2 = pd.DataFrame(data=sst_anomaly.data, index=sst.time.data, columns=["SSTa (C)"])

df2.index.name = "Date"
df2.hvplot.area(x="Date", y="SSTa (C)", grid=True, title="SST Anomalies")

***
## We analyze the data further by choosing a threshold.

- One way is to set a threshold that has some relevance.  For example, a thermal threshold for a marine species we are studying. 

- Another way is choosing the maximum value in the climatology (mean value + 1 standard deviation), which we can calculate or read by hovering our cursor over the climatology plot above.

### Once the threshold is choosen, we identify when SST is over that threshold, and count how many days that occurred each year

In [ ]:
# first, we define a function that take a threshold value, and analyze and plot our data
def SST_above(thr):

    # first part - values above threshold
    # first plot the timeseries
    plt.figure(figsize=(8, 4))
    plt.plot(sst.time, sst.data, lw=1)
    a = (
        sst >= thr
    )  # test when data is equal or greater than the threshold. a is a logical array (True/False values)
    plt.plot(
        sst.time[a], sst.data[a], ".r", markersize=3
    )  # plot only the values equal or above threshold
    # all stuff here to make it look good
    plt.ylabel("SST ($^\circ$C)")
    plt.xlabel("Year")
    plt.title("Location: " + str(latr) + "$^\circ$N, " + str(lonr) + "$^\circ$W")
    plt.grid(True, alpha=0.3)
    plt.show()  # display and finaiize this figure, so the next is not overwritten

    # second part - days per year above threshold
    dts = sst[sst >= thr].time  # select dates when SST is equal or greater than the threshold. note that this time is not a logical array, but the time values
    hot_days = dts.groupby("time.year").count()  # agregate by year, by counting
    plt.bar(hot_days.year, hot_days)  # bar plot of days per year
    plt.xlim(int(dater[0][:4]), int(dater[1][:4]) + 1)  # make it nice
    plt.ylabel("No. days above " + str(np.round(thr, 1)) + "C")
    plt.grid(True, alpha=0.3)
    plt.show()  # display


## Second, the actual analuysis: two examples ##
### Maximum climatology threshold
thr = df["+Std"].max()  # setting threshold as maximum climatological value: mean + 1 standard deviation
print("Max climatological SST = ", np.round(thr, 1), "C")
SST_above(thr)  # Call function we defined

### A relevant threshold.
# For example, for hawaii (the select region), 28C is a relevant threshold for coral bleaching (https://coralreefwatch.noaa.gov/product/5km/tutorial/crw08a_bleaching_threshold.php)
thr = 28
print("\n\nBiologically relevant SST = ", thr, "C")
SST_above(thr)  # Call function

***
#### Now, a different analsys of anomalously warm SST days. 
## Marine heatwaves
Defined as any period with SST anomalies above the threshold set by the 90th percentile value of a given period SST anomalies - in this case our data time period.

In [ ]:
# First, calculate the threshold: 90th percentile
thr = np.percentile(sst_anomaly, 90)

# Same plot as in our function above, but this time we are plotting the anomalies.
plt.figure(figsize=(8, 4))
plt.plot(sst_anomaly.time, sst_anomaly.data, lw=1)
plt.axhline(y=0, c="k", zorder=0, alpha=0.5)  # add a line to highlight the x axis

a = sst_anomaly >= thr  # select data above the threshold
plt.plot(sst_anomaly.time[a], sst_anomaly.data[a], ".r", markersize=3)
# all stuff here to make it look good
plt.ylabel("SST anomalies ($^\circ$C)")
plt.xlabel("Year")
plt.title("Location: " + str(latr) + "$^\circ$N, " + str(lonr) + "$^\circ$W")
plt.grid(True, alpha=0.3)
plt.show()

# Now plot on the original data (not anomalies)
plt.figure(figsize=(8, 4))
plt.plot(sst.time, sst.data, lw=1)
plt.plot(
    sst.time[a], sst.data[a], ".r", markersize=3
)  # plot only the values equal or above threshold
# all stuff here to make it look good
plt.ylabel("SST ($^\circ$C)")
plt.xlabel("Year")
plt.title("Location: " + str(latr) + "$^\circ$N, " + str(lonr) + "$^\circ$W")
plt.grid(True, alpha=0.3)
plt.show()

# Plot of marine heatwave days  per year
dts = sst_anomaly[sst_anomaly >= thr].time
mhw = dts.groupby("time.year").count()
plt.bar(mhw.year, mhw)
plt.ylabel("No. days SSTa > " + str(np.round(thr, 1)) + "C")
plt.grid(True, alpha=0.3)
plt.show()

mhw  # print the numbers of days

***
## Finally, let's explore the SST field around our selected point or region durring the warmest day or coldest day.

In [ ]:
# Find out max and min SST values and the date when they occur
minv = sst.min()  # find mininum SST value
mindate = sst[sst == minv].time.data  # find when this min value occurred
maxv = sst.max()  # find maximum SST value
maxdate = sst[sst == maxv].time.data  # find when the max value occurred

In [ ]:
# define a function that go back to the SST data in the cloud, but we now load a different subset
# an specific day, but now always a region: the region selected or a region around the selected point
def select_area(day):  # the function input is a day
    if (latr[0] == latr[1]) | (lonr[0] == lonr[1]):  # if input data was one point
        sst2 = ds_sst.sel(
            time=day, lat=slice(latr - 2, latr + 2), lon=slice(lonr - 2, lonr + 2)
        ).load()
    else:  # if input data was a region
        sst2 = ds_sst.sel(
            time=day, lat=slice(latr[0], latr[1]), lon=slice(lonr[0], lonr[1])
        ).load()
    sst3 = sst2["analysed_sst"] - 273.15
    mask = sst2["mask"].where(sst2["mask"] < 2)
    sst3 = sst3 * mask
    return sst3  # returns the data array of the region at the given date (the region is the defined at the beginning of the script)

In [ ]:
# plot warmest day
msst = select_area(maxdate)  # call function with day of warmest SST
msst.hvplot.quadmesh(
    x="lon",
    y="lat",
    coastline=True,
    clabel="T [C]",
    cmap="coolwarm",
    title=str(maxdate[0])[:10],
)
# this image plot also gives you extra information when you hover your cursor around

In [ ]:
# plot coolest day
msst = select_area(mindate)  # call function with day of coolest SST
msst.hvplot.quadmesh(
    x="lon",
    y="lat",
    coastline=True,
    clabel="T [C]",
    cmap="coolwarm",
    title=str(mindate[0])[:10],
)

***
# Resources

For the cloud and data in the cloud, see resources listed in Chapter 5.

Resources specifically for this chapter:
- [MUR SST Data](https://registry.opendata.aws/mur/). SST data in the cloud, with references the official datta website, examples and other resources.
- [Pangeo OSM2020 Tutorial](https://github.com/pangeo-gallery/osm2020tutorial). This is a very good tutorial for ocean application and cloud computing. Plenty of examples. Many of the commands here are from this tutorial.

### About MHW

- [Marine heatwaves](http://www.marineheatwaves.org/all-about-mhws.html). A good place to begin to get info about the subject.
- [Marine heatwaves code](https://github.com/ecjoliver/marineHeatWaves). Marine heatwaves code from E. Oliver. 

### If you want to learn more:
- [Methods for accessing a AWS bucket](https://docs.aws.amazon.com/AmazonS3/latest/userguide/access-bucket-intro.html). Bucket is the name of the cloud storage object. S3 stands for Amazon's Simple Storage Service.
- [hvplot site](https://hvplot.holoviz.org/index.html). Plotting tool used here.
- [zarr](https://zarr.readthedocs.io/en/stable/) Big data storage formatt
